# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
# this increases the degree of parallelism and the memory footprint
"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)"
# doing some data transformation
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)"

Note that increasing the degree of parallelism also increases the memory footprint.

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
#fit the model
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10, verbose = False)

Find what features had non-zero weight.

In [6]:
# find the model coefficients
coeff = model_all.coefficients
coeff.print_rows(num_rows=18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910107  |  None  |
|   sqft_living    |  None | 24.420720982445495 |  None  |
| sqft_living_sqrt |  None | 350.0605533860576  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976041  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [7]:
# answer to the quiz question
choosen = coeff[coeff['value'] > 0] # create a subset of the sframe
choosen.print_rows(num_rows = 18) # set to 18 to consider all features

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|    bathrooms     |  None | 8468.531086910107  |  None  |
|   sqft_living    |  None | 24.420720982445495 |  None  |
| sqft_living_sqrt |  None | 350.0605533860576  |  None  |
|      grade       |  None | 842.0680348976041  |  None  |
|    sqft_above    |  None | 20.02472241709115  |  None  |
+------------------+-------+--------------------+--------+
[6 rows x 4 columns]



# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import pprint
import numpy as np
val_err_pred = {} # to store the error for each penalty
for penalty in np.logspace(1,7, num=13):
    model = turicreate.linear_regression.create(training, features=all_features, target='price', validation_set=None, 
                                               verbose = None, l1_penalty=penalty, l2_penalty=0)
    prediction = model.predict(validation)
    residuals = validation['price'] - prediction
    RSS_on_val = (residuals**2).sum()
    val_err_pred[penalty] = RSS_on_val
pprint.pprint(val_err_pred)
print("")
print("Now, I am done estimating the errors")

{10.0: 625766285142460.5,
 31.622776601683793: 625766285362393.9,
 100.0: 625766286057885.1,
 316.22776601683796: 625766288257224.8,
 1000.0: 625766295212186.1,
 3162.2776601683795: 625766317206080.5,
 10000.0: 625766386760658.1,
 31622.776601683792: 625766606749278.4,
 100000.0: 625767302791635.2,
 316227.7660168379: 625769507643885.8,
 1000000.0: 625776517727024.2,
 3162277.6601683795: 625799062845466.6,
 10000000.0: 625883719085425.2}

Now, I am done estimating the errors


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [10]:
# The best value is the one that most minimize rss
print(min(val_err_pred.items(), key=lambda x: x[1]))

(10.0, 625766285142460.5)


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [11]:
#fit the model
best_penalty = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=10.0, verbose = False)

In [12]:
coeff = best_penalty.coefficients
#coeff.print_rows(num_rows=20)
non_zero_weights = coeff[coeff['value'] > 0]
non_zero_weights.print_rows(num_rows=20)

+------------------+-------+-----------------------+--------+
|       name       | index |         value         | stderr |
+------------------+-------+-----------------------+--------+
|   (intercept)    |  None |    18993.4272127706   |  None  |
|     bedrooms     |  None |   7936.967679031298   |  None  |
| bedrooms_square  |  None |    936.993368193299   |  None  |
|    bathrooms     |  None |   25409.588934120657  |  None  |
|   sqft_living    |  None |   39.11513637970765   |  None  |
| sqft_living_sqrt |  None |   1124.6502128077168  |  None  |
|     sqft_lot     |  None | 0.0034836182229897304 |  None  |
|  sqft_lot_sqrt   |  None |   148.2583910114082   |  None  |
|      floors      |  None |   21204.335466950117  |  None  |
|  floors_square   |  None |   12915.524336072429  |  None  |
|    waterfront    |  None |   601905.5945452718   |  None  |
|       view       |  None |   93312.85731187186   |  None  |
|    condition     |  None |   6609.035712447213   |  None  |
|      g

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
non_zero_weight = {}
for l1_penalty in np.logspace(8, 10, num=20):
    model = turicreate.linear_regression.create(training, features = all_features, target = 'price', l1_penalty=l1_penalty,
                                               l2_penalty=0., validation_set=None, verbose=False)
    non_zero_weight[l1_penalty] = model.coefficients['value'].nnz()
pprint.pprint(non_zero_weight)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [18]:
val_err_pred2 = {} # store the value inside the dictionary
# search the space of possible penalties
for l1_penalty in np.linspace(l1_penalty_min, l1_penalty_max, 20):
    #fit the model
    model = turicreate.linear_regression.create(training, features = all_features, target ='price', validation_set=None,
                                               verbose = False, l1_penalty=l1_penalty, l2_penalty=0.)
    # Make prediction on validation set
    prediction = model.predict(validation)
    # calculate the residuals
    residuals = validation['price'] - prediction
    RSS_on_val = (residuals**2).sum()
    # here, we store two values in our dictionary as evident in the printed format
    val_err_pred2[l1_penalty] = RSS_on_val, model.coefficients['value'].nnz() 
pprint.pprint(val_err_pred2)
print("")
print("Now, I am done estimating the errors")

{2976351441.6313133: (966925692362084.9, 10),
 3019316638.9524155: (974019450084554.5, 10),
 3062281836.2735176: (981188367942453.5, 10),
 3105247033.5946198: (989328342459473.2, 10),
 3148212230.915722: (998783211265889.4, 10),
 3191177428.236824: (1008477167020093.5, 10),
 3234142625.557926: (1018298780553819.6, 10),
 3277107822.8790283: (1028247992205977.0, 10),
 3320073020.2001305: (1034616909232827.2, 8),
 3363038217.5212326: (1038554735941040.1, 8),
 3406003414.8423347: (1043237237871701.8, 8),
 3448968612.163437: (1046937488751710.9, 7),
 3491933809.484539: (1051147625612862.5, 7),
 3534899006.805641: (1055992735342999.4, 7),
 3577864204.126743: (1060799531763287.4, 7),
 3620829401.447845: (1065707689498229.2, 6),
 3663794598.768947: (1069464335425585.1, 6),
 3706759796.0900493: (1073504549585599.2, 6),
 3749724993.4111514: (1077632775581415.1, 6),
 3792690190.7322536: (1081867592324112.1, 6)}

Now, I am done estimating the errors


In [19]:
lowest = {} # store the data in the dictionary
# k represents the key and 'v' represents the value
for k, v in val_err_pred2.items():
    # v[1] is the second index of values
    if v[1] == max_nonzeros: # i.e if it is equal to 7
        lowest[k] = v[0] # get the penalty(i.e the keys) and the RSS (i.e the values)
pprint.pprint(lowest)
#find the lowest RSS
print("")
print("This has the lowest RSS and with max_nonzeros equal to 7")
print(min(lowest.items(), key = lambda x: x[1]))

{3448968612.163437: 1046937488751710.9,
 3491933809.484539: 1051147625612862.5,
 3534899006.805641: 1055992735342999.4,
 3577864204.126743: 1060799531763287.4}

This has the lowest RSS and with max_nonzeros equal to 7
(3448968612.163437, 1046937488751710.9)


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [20]:
last_penalty = 3448968612.163437
last_model = turicreate.linear_regression.create(training, features = all_features, target='price', validation_set=None,
                                                l1_penalty=last_penalty, l2_penalty=0., verbose=False)
last_coef = last_model.coefficients

In [21]:
lastlast = last_coef[last_coef['value'] > 0]
lastlast.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.1925443279  |  None  |
|     bedrooms     |  None | 661.7227177822257  |  None  |
|    bathrooms     |  None | 15873.957259267983 |  None  |
|   sqft_living    |  None | 32.410221451259716 |  None  |
| sqft_living_sqrt |  None | 690.1147733133171  |  None  |
|      grade       |  None | 2899.4202697498836 |  None  |
|    sqft_above    |  None | 30.011575302201155 |  None  |
+------------------+-------+--------------------+--------+
[7 rows x 4 columns]

